In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore',category=FutureWarning)

covid_df = pd.read_csv("us_counties_covid19_daily.csv")
covid_df = covid_df.drop(['fips','deaths'],1)
covid_df.head()

,date,county,state,cases
0,2020-01-21,Snohomish,Washington,1
1,2020-01-22,Snohomish,Washington,1
2,2020-01-23,Snohomish,Washington,1
3,2020-01-24,Cook,Illinois,1
4,2020-01-24,Snohomish,Washington,1


In [2]:
# Select data for each city
states = ['Illinois','Texas','Arizona','Pennsylvania','Texas','California','Texas','Florida','Ohio','Indiana']
counties = ['Cook','Harris','Maricopa','Philadelphia','Bexar','San Diego','Travis','Duval','Franklin','Marion']
cities = ['Chicago','Houston','Phoenix','Philadelphia','San Antonio','San Diego','Austin','Jacksonville','Columbus','Indianapolis']
state_county = list(zip(states,counties,cities))

In [3]:
# Process data for each city

# Create new DataFrame to bring data for each city together
city_covid_df = pd.DataFrame()

for state, county, city in state_county:
    # Get the correct county data and add city column
    city_df = covid_df[(covid_df['state'] == state) & (covid_df['county'] == county)].reset_index().drop(['index'],1)
    city_df['city'] = city

    # new_cases column
    # record the number of new cases for today
    city_df['new_cases'] = pd.Series(dtype=int)

    for i in range(1,len(city_df.index)):
        current_cases = city_df.iloc[i,3]
        previous_cases = city_df.iloc[i-1,3]

        city_df.iloc[i,5] = current_cases - previous_cases

    # delta7 column
    # record the change in number of cases from 7 days ago to today
    city_df['delta7'] = pd.Series(dtype=int)

    for i in range(7, len(city_df.index)):
        current_cases = city_df.iloc[i,3]
        previous_cases = city_df.iloc[i-7,3]

        city_df.iloc[i,6] = current_cases - previous_cases
    
    # delta14 column
    # record the change in number of cases from 7 days ago to today
    city_df['delta14'] = pd.Series(dtype=int)

    for i in range(14,len(city_df.index)):
        current_cases = city_df.iloc[i,3]
        previous_cases = city_df.iloc[i-7,3]

        city_df.iloc[i,7] = current_cases - previous_cases

    # Add to final_df
    city_covid_df = pd.concat([city_covid_df,city_df])

city_covid_df.head(30)

,date,county,state,cases,city,new_cases,delta7,delta14
0,2020-01-24,Cook,Illinois,1,Chicago,NaN,NaN,NaN
1,2020-01-25,Cook,Illinois,1,Chicago,0.0,NaN,NaN
2,2020-01-26,Cook,Illinois,1,Chicago,0.0,NaN,NaN
3,2020-01-27,Cook,Illinois,1,Chicago,0.0,NaN,NaN
4,2020-01-28,Cook,Illinois,1,Chicago,0.0,NaN,NaN
5,2020-01-29,Cook,Illinois,1,Chicago,0.0,NaN,NaN
6,2020-01-30,Cook,Illinois,2,Chicago,1.0,NaN,NaN
7,2020-01-31,Cook,Illinois,2,Chicago,0.0,1.0,NaN
8,2020-02-01,Cook,Illinois,2,Chicago,0.0,1.0,NaN
9,2020-02-02,Cook,Illinois,2,Chicago,0.0,1.0,NaN


In [4]:
# Convert float values to int
city_covid_df = city_covid_df.convert_dtypes()
city_covid_df.head(30)

# put columns in correct order
city_covid_df = city_covid_df[['date','city','cases','new_cases','delta7','delta14']]

# rename 'cases' to 'total_cases'
city_covid_df = city_covid_df.rename(columns={'cases':'total_cases'})

city_covid_df.head(30)

,date,city,total_cases,new_cases,delta7,delta14
0,2020-01-24,Chicago,1,<NA>,<NA>,<NA>
1,2020-01-25,Chicago,1,0,<NA>,<NA>
2,2020-01-26,Chicago,1,0,<NA>,<NA>
3,2020-01-27,Chicago,1,0,<NA>,<NA>
4,2020-01-28,Chicago,1,0,<NA>,<NA>
5,2020-01-29,Chicago,1,0,<NA>,<NA>
6,2020-01-30,Chicago,2,1,<NA>,<NA>
7,2020-01-31,Chicago,2,0,1,<NA>
8,2020-02-01,Chicago,2,0,1,<NA>
9,2020-02-02,Chicago,2,0,1,<NA>


In [5]:
len(city_covid_df.index)

2856

In [6]:
city_covid_df.to_csv("city_covid.csv")